# Unix Class Tasks — Set 03 with Answers (Bash)


## 1) Generate a new SSH key and change its passphrase
Create an **ed25519** SSH key named `id_remote_ed25519` (with a comment `class03`) and then change its passphrase.
- Hint: `ssh-keygen -t ed25519 -C 'class03' -f id_remote_ed25519`
- Then: change the passphrase for this key.

In [ ]:
%%bash
## Generate an ed25519 key (modern default for new keys) with a helpful comment
ssh-keygen -t ed25519 -C 'class03' -f id_remote_ed25519

## Change passphrase for the key (you can press Enter for old passphrase if none)
ssh-keygen -p -f id_remote_ed25519

## 2) Add key to ssh-agent and create a Host alias
Start `ssh-agent`, add your new key, and create a minimal `~/.ssh/config` entry for an alias **uni04**
that points to host `fse4ai_04_server` with user `student` and uses your `id_remote_ed25519` key.
After creating the alias, test it with a dry connection (e.g., `ssh -o BatchMode=yes -F ~/.ssh/config uni04 true`).

In [ ]:
%%bash
## Start agent (if not already running) and add the key
eval "$(ssh-agent -s)"
ssh-add ./id_remote_ed25519

## Create a minimal SSH config entry (idempotent append)
mkdir -p ~/.ssh && chmod 700 ~/.ssh
cat >> ~/.ssh/config <<'EOF'
Host uni04
    HostName fse4ai_04_server
    User student
    IdentityFile ~/id_remote_ed25519
    IdentitiesOnly yes
EOF
chmod 600 ~/.ssh/config

## Dry connection using config; BatchMode avoids password prompts
ssh -o BatchMode=yes uni04 true || true
# NOTE: If host keys are unknown, first run: ssh -o StrictHostKeyChecking=accept-new uni04 true

## 3) Copy public key to the server (dry run + real)
Do a **dry-run** of copying your public key to the remote server using the alias from task 2, then perform the real copy.
- Dry-run: `ssh-copy-id -n -i id_remote_ed25519.pub uni04`
- Real: `ssh-copy-id -i id_remote_ed25519.pub uni04`

In [ ]:
%%bash
## Dry run (shows what would be done)
ssh-copy-id -n -i id_remote_ed25519.pub uni04

## Real copy (may prompt for the remote password once)
ssh-copy-id -i id_remote_ed25519.pub uni04

## 4) Run a long job remotely with nohup (two ways)
Launch `sleep 12; echo done` on the remote in the background with `nohup`, capturing stdout to `std.out` and stderr to `std.err`, then disconnect.
a) Connect first, run commands, then `exit`.
b) Do it in a **single** one‑liner using `ssh -f`.

In [ ]:
%%bash
## (a) Interactive approach
ssh uni04 <<'REMOTE'
nohup bash -c 'sleep 12; echo done' > std.out 2> std.err &
exit
REMOTE

## (b) Single-shot non-interactive; -f backgrounds ssh after auth
ssh -f uni04 "nohup bash -c 'sleep 12; echo done' > std.out 2> std.err &"

## 5) Redirect streams and mirror them to files
From the **client** container, redirect the output of `ls -lh` so that stdout goes to stderr, and then use `tee` to both display and save outputs:
1) Send stdout to stderr.
2) Save stdout to `std.out` and stderr to `std.err` while still showing stdout live.
3) Use **process substitution** so both streams are displayed and saved into their files.

In [ ]:
%%bash
## 1) stdout -> stderr
ls -lh 1>&2

## 2) Save stdout while showing it; stderr to file
ls -lh 2> std.err | tee std.out

## 3) Process substitution (Bash feature): duplicates streams cleanly
ls -lh > >(tee -a std.out) 2> >(tee -a std.err >&2)

## 6) rsync with size window and excludes (from remote logs/)
From the **client**, copy files from remote `~/logs` to local `logs_local/` **only** if they are between 2KB and 2MB.
Exclude files whose names contain `tmp` and show progress.

In [ ]:
%%bash
mkdir -p logs_local
rsync -av --progress \
  --min-size=2K --max-size=2M \
  --exclude='*tmp*' \
  uni04:logs/ ./logs_local/

## 7) Mirror a directory (deletes local extras)
Make a **mirror** of remote `~/logs/` into local `logs_mirror/` so that files deleted on remote are also deleted locally.
Show itemized changes.

In [ ]:
%%bash
mkdir -p logs_mirror
rsync -av --progress --delete --itemize-changes uni04:logs/ ./logs_mirror/

## 8) Copy a single file using scp with explicit options
Copy `~/logs/server.log` from the remote to the current directory using `scp`.
Use `-o IdentitiesOnly=yes` to force your key and `-o StrictHostKeyChecking=accept-new` on first connect.

In [ ]:
%%bash
## Note: OpenSSH 9+ uses SFTP-mode scp by default (safer). Options are passed through to SSH.
scp -o IdentitiesOnly=yes -o StrictHostKeyChecking=accept-new uni04:logs/server.log ./